In [1]:
import serial
import mysql.connector
import os
import re
import face_recognition
import cv2
import numpy as np
import time

In [6]:
ser = serial.Serial('COM3',9600)
mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "",
        database = "ASPS",
)
myCourser = mydb.cursor()
#14388655

In [13]:
line = str(ser.readline())
userPassword = "".join( re.findall(r'[0-9]' , line))
if SelectUser(userPassword) != 0 :
    res = SelectUser(userPassword)
    email = res[2]
    images = InnerJoin(email , userPassword)
    encodeListKnown = findEncodeings(images)
    Image_recg(encodeListKnown , email)
else:
    flag = '0' 
    ser.write(flag.encode('utf-8'))


In [3]:
########## function TO check pass(pass)############
def SelectUser(passArd):
    sql = "SELECT * FROM `users` WHERE user_pass_ard = %s"
    adr = (passArd,)
    myCourser.execute(sql,adr)
    myresult = myCourser.fetchone()
    if myresult != None:
        return myresult
    else: 
        return 0

############### function TO Get Image of User  #################
def InnerJoin(email , passArd):
    sql = "SELECT image_path FROM `users` INNER JOIN `images` ON email_user = %s WHERE user_pass_ard = %s"
    adr = (email,passArd)
    myCourser.execute(sql,adr)
    myresult = myCourser.fetchall()
    Userimg = []
    for img in myresult:
        pic = "".join(img)
        personimg = cv2.imread(f'../uploads/{pic}')
        Userimg.append(personimg)
    return Userimg



def findEncodeings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def Image_recg(encodeListKnown , email):
    cap = cv2.VideoCapture(0)
    while True :
        _, img = cap.read()
        if img.size == 0:
            break
        imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        faceCurentFrame = face_recognition.face_locations(imgS)
        encodeCurentFrame = face_recognition.face_encodings(imgS, faceCurentFrame)
        for encodeface, faceLoc in zip(encodeCurentFrame, faceCurentFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeface)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeface)
            matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.rectangle(img, (x1,y2-35), (x2,y2), (0,0,255), cv2.FILLED)
            cv2.putText(img, email, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
#             InsertUser(email)
            flag = '1'
            ser.write(flag.encode('utf-8'))
#             time.sleep(5)
#             break
        else:
            flag = '0'
            ser.write(flag.encode('utf-8'))
#             cv2.imwrite('captured_image.jpg', img)
        cv2.imshow('Face Recogntion', img)
    cap.release()
    cv2.destroyAllWindows()
    
##################### attendance #######################
def InsertUser(email):
    sql = "INSERT INTO `attendance` (email_user , id_place) VALUES (%s, %s)"
    adr = (email,1)
    myCourser.execute(sql,adr)
    myCourser.rowcount 
    mydb.commit()
       
    

In [99]:
image = face_recognition.load_image_file(f'../uploads/64511060c0cab.jpg')
biden_encoding = face_recognition.face_encodings(image)[0]
Unkown_image = face_recognition.load_image_file(f'../uploads/64511060d8b77.jpg')
person = face_recognition.face_encodings(Unkown_image)[0]
results = face_recognition.compare_faces([biden_encoding], person)
results

[True]

In [102]:
curPersonn = cv2.imread(f'../uploads/64511060c0cab.jpg')
biden_encoding = face_recognition.face_encodings(curPersonn)[0]
unkown = cv2.imread(f'../uploads/64511060d8b77.jpg')
person = face_recognition.face_encodings(unkown)[0]
res = face_recognition.compare_faces([biden_encoding], person)
res                                        

[True]